In [5]:
import torch
from torch import nn

class MLP(nn.Module):

    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256,10)

    def forward(self,x):
        o = self.act(self.hidden(x))
        return self.output(o)

In [ ]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)

In [ ]:
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self,**kwargs):
        super(MyLayer,self).__init__(**kwargs)

    def forward(self,x):
        return x - x.mean()   

layer = MyLayer()
x = torch.tensor([1,2,3,4,5],dtype = torch.float)
layer(x)

In [ ]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense,self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))

    def forward(self,x):
        for i in range(len(self.params)):
            x = torch.mm(x,self.params[i])
        return x
net = MyListDense()
print(net)

In [ ]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params = nn.ParameterDict({
            'linear1' : nn.Parameter(torch.randn(4,4)),
            'linear2' : nn.Parameter(torch.randn(4,1))
        })
        self.params.update({
            'linear3':nn.Parameter(torch.randn(4,2))
        })
    
    def forward(self,x,choice = 'linear1'):
        return torch.mm(x,self.params[choice])
net = MyDictDense()
print(net)


In [ ]:
import torch
from torch import nn

def corr2d(X,K):
    h,w = K.shape
    X,K = X.float(),K.float()
    Y = torch.zeros((X.shape[0] - h +1,X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i + h,j:j + w] * K).sum()
    return Y

class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size,kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self,x):
        return corr2d(x,self.weight) + self.bias
    
X = torch.tensor([[1, 1, 1, 1, 1] for i in range(5)])
conv2D = Conv2D(3)
output = conv2D(X)
print(output)


In [ ]:


def comp_conv2d(conv2d,X):
    X = X.view((1,1)+X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])



conv2d = nn.Conv2d(in_channels = 1,out_channels = 1,kernel_size = 3,padding = 1)
X = torch.rand(8,8)
comp_conv2d(conv2d,X).shape

In [ ]:
conv2d = nn.Conv2d(in_channels = 1, out_channels = 1,kernel_size = (5,3),padding = (2,1))
comp_conv2d(conv2d,X).shape

In [ ]:
conv2d = nn.Conv2d(1,1,kernel_size = (3,5),padding = (0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

In [ ]:
import torch
from torch import nn

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    
    for i in range(Y.shape[0]):  # Adjusted range
        for j in range(Y.shape[1]):  # Adjusted range
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

X = torch.arange(9, dtype=torch.float).reshape(3, 3)
pool2d(X, (2, 2), mode='avg')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)

        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(-1,int(x.numel()/x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


net = Net()
x = torch.rand(1,1,32,32)
print(net)



In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

In [86]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[0.0000, 0.0330, 0.0777, 0.0170, 0.0000, 0.0644, 0.0000, 0.1316, 0.0000,
         0.0653]], grad_fn=<ReluBackward0>)


In [88]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [95]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,96,11,4),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96,256,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,256,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )

        # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,10)
        )
    
    def forward(self,img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0],-1))
        return output

In [96]:
net = AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (